In [37]:
import openai
import elasticsearch

In [38]:
import json
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
with open('./documents.json', 'rt') as f_in:
    documents_all =json.load(f_in)
    

--2024-06-25 13:05:46--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-06-25 13:05:46--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.6’

documents.json.6    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-25 13:05:46 (51.5 MB/s) - ‘documents.json.6’ saved [65

In [39]:
    documents_all[0].keys()

dict_keys(['course', 'documents'])

In [40]:
documents = []
for course in documents_all:
    for doc in course['documents']:
        doc['course']= course['course']
        documents.append(doc)

In [41]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

## Run elastic search client
````
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
````

In [42]:
#creating elastic serach plan
from elasticsearch import Elasticsearch
es_client = Elasticsearch(hosts=['http://localhost:9200'])
index_name = "course-questions"
# es_client.info()

In [43]:
# index_settings = {
#     "settings": { #settings are importants when you have clusters
#         "number_of_shards": 1,
#         "number_of_replicas": 0
#     },
#     "mappings": {
#         "properties": {
#             "text": {"type": "text"},
#             "section": {"type": "text"},
#             "question": {"type": "text"},
#             "course": {"type": "keyword"} 
#         }
#     }
# }

# index_name = "course-questions"
# response = es_client.indices.create(index=index_name, body=index_settings)
# #I think thisis  only ran once
# response

In [44]:
from tqdm.auto import tqdm

In [45]:
#save docments in our es client
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 41.52it/s]


In [46]:
query = "I just discovererd the course. Can I still join?"

In [47]:
#turning into an elastic query
search_query = {
    "size": 5, #limit kinda thing
    "query": {
        "bool": { #type of query, history. two parts must (or should)  and filter
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"], #I wanna consider alll of them, my keywords can bein any of these. ^3 means that that field is 3 times more important
                    "type": "best_fields"
                }
            },
            "filter": { #we filter, so we only consider documents beonging to thiscouts
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [48]:
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 499, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 798, 'relation': 'eq'}, 'max_score': 73.286255, 'hits': [{'_index': 'course-questions', '_id': 'lJhrT5ABno94xw0RQ1HR', '_score': 73.286255, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'SJiAT5ABno94xw0RqlVV', '_score': 73.286255, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [49]:
response['hits']

{'total': {'value': 798, 'relation': 'eq'},
 'max_score': 73.286255,
 'hits': [{'_index': 'course-questions',
   '_id': 'lJhrT5ABno94xw0RQ1HR',
   '_score': 73.286255,
   '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
    'section': 'General course-related questions',
    'question': 'Course - Can I still join the course after the start date?',
    'course': 'data-engineering-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'SJiAT5ABno94xw0RqlVV',
   '_score': 73.286255,
   '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
    'section': 'General course-related questions',
    'question': 'Course - Can I still join

In [50]:
#We can clean it by removing the index, id,score....
relevant_docs = []
for hit in response['hits']['hits']:
    doc = hit['_source']
    relevant_docs.append(doc)
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - What can I do before the course starts?
Answer: You can start by installing and setting up all the dependenc...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...

Section: General course-related questions
Question: Course - What can I do before the course starts?
Answer: You can start by installing and setting up all the dependenc...



In [51]:
def retrieve(query):
    search_query = {
    "size": 5, #limit kinda thing
    "query": {
        "bool": { #type of query, history. two parts must (or should)  and filter
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"], #I wanna consider alll of them, my keywords can bein any of these. ^3 means that that field is 3 times more important
                    "type": "best_fields"
                }
            },
            "filter": { #we filter, so we only consider documents beonging to thiscouts
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
    }
    response = es_client.search(index=index_name, body=search_query)
    relevant_docs = []
    for hit in response['hits']['hits']:
        doc = hit['_source']
        relevant_docs.append(doc)
    return relevant_docs

In [52]:
context_documents = retrieve(query)
context_documents

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and 

In [53]:
from openai import OpenAI

In [54]:
openai_client = OpenAI(api_key='ollama', base_url='http://localhost:11434/v1')

In [55]:
context_template = """
Section: {section}
Question: {question}
Answer: {text}
""".strip()

prompt_template = """
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()


def build_context(documents):
    context_result = ""
    
    for doc in documents:
        doc_str = context_template.format(**doc)
        context_result += ("\n\n" + doc_str)
    
    return context_result.strip()


def build_prompt(user_question, documents):
    context = build_context(documents)
    prompt = prompt_template.format(
        user_question=user_question,
        context=context
    )
    return prompt

In [56]:
def llm(prompt):
    oai_response = openai_client.chat.completions.create(
        model='phi3',
        messages = [{"role": "user", "content": prompt}])
    answer = oai_response.choices[0].message.content
    return answer

In [57]:
print(build_prompt(query, context_documents))

You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: I just discovererd the course. Can I still join?

CONTEXT:

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - What can I do befo

In [58]:
#there are 3 steps
def rag(query):
    context_documents = retrieve(query) #1.get all relevant docs
    prompt = build_prompt(query, context_documents) #2.prompt llm with our relevant/ context_douents
    response = llm(prompt)
    return response

In [59]:
rag(query)

" Yes, even if you didn't register beforehand, you can still join the course as long as you meet any necessary requirements outlined in the course materials. However, please be aware of deadlines for submitting homeworks and final projects, so it is advisable not to leave everything until the last minute.\n\nAdditionally, if you're planning on joining after the start date, make sure to install and set up all necessary dependencies and requirements such as a Google Cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git before beginning your coursework. Familiarize yourself with prerequisites and syllabus to ensure you're prepared for the subjects covered in the course."